<a href="https://colab.research.google.com/github/mprksa/kubus5/blob/main/DT_TestRandom.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [57]:
!git clone https://github.com/mprksa/kubus5.git

Cloning into 'kubus5'...
remote: Enumerating objects: 39, done.
remote: Counting objects: 100% (39/39), done.
remote: Compressing objects: 100% (37/37), done.
remote: Total 39 (delta 9), reused 0 (delta 0), pack-reused 0
Receiving objects: 100% (39/39), 24.55 MiB | 25.19 MiB/s, done.
Resolving deltas: 100% (9/9), done.


In [75]:
import cv2
import joblib
import numpy as np

# Fungsi untuk memuat model
def load_model(model_filename):
    with open(model_filename, 'rb') as model_file:
        dt, scaler, _ = joblib.load(model_file)  # Hanya memuat dt dan scaler, label_encoder tidak digunakan lagi
    return dt, scaler

# Fungsi untuk melakukan deteksi objek menggunakan model dan ruang warna HSV
def detect_objects(img, model):
    dt, scaler = model

    # Ubah gambar menjadi ruang warna HSV
    img_hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)

    # Resize gambar ke ukuran yang sesuai dengan scaler
    target_size = (256, 256)  # Misalnya, ukuran yang sesuai dengan scaler
    img_resized = cv2.resize(img_hsv, target_size)

    img_flat = img_resized.flatten().reshape(1, -1)

    # Normalisasi fitur
    img_normalized = scaler.transform(img_flat)

    # Prediksi menggunakan model Decision Tree
    prediction = dt.predict(img_normalized)[0]
    probabilities = dt.predict_proba(img_normalized)[0]  # Peroleh probabilitas prediksi untuk semua kelas

    return prediction, probabilities

In [85]:
if __name__ == '__main__':
    # Load model yang telah disimpan
    model_filename = 'decision_tree_model.pkl'
    model = load_model(model_filename)
    # Daftar path gambar yang akan diproses
    image_paths = ['kubus5/w/Biru Muda.JPG', 'kubus5/w/Biru Tua.JPG', 'kubus5/w/Coklat.JPG', 'kubus5/w/Hijau Muda.JPG', 'kubus5/w/Hijau Tua.JPG', 'kubus5/w/Kuning Muda.JPG', 'kubus5/w/Orange.JPG', 'kubus5/w/Pink.JPG', 'kubus5/w/Ungu.JPG']  # Ubah sesuai dengan gambar yang ingin Anda deteksi


In [86]:
class_labels = {
    0: "Biru",
    1: "Hitam",
    2: "Kuning",
    3: "Merah",
    4: "Putih"
}

total_correct = 0
total_images = len(image_paths)

for image_path in image_paths:
    # Load gambar untuk deteksi objek
    img = cv2.imread(image_path)

    if img is not None:
        # Deteksi objek dalam gambar
        prediction, probabilities = detect_objects(img, model)

        # Cetak hasil prediksi
        predicted_label = class_labels[prediction]
        accuracy = probabilities[prediction] * 100  # Hitung akurasi prediksi

        print(f"Deteksi Objek pada {image_path}: Prediksi = {predicted_label} ({accuracy:.2f}%)")
        # Periksa jika prediksi benar
        if predicted_label == image_path.split('.')[0].split('_')[0].capitalize().lower():
            total_correct += 1 # This line was incorrectly indented
    else:
        print(f"Gagal memuat gambar: {image_path}")

Deteksi Objek pada kubus5/w/Biru Muda.JPG: Prediksi = Biru (100.00%)
Deteksi Objek pada kubus5/w/Biru Tua.JPG: Prediksi = Biru (100.00%)
Deteksi Objek pada kubus5/w/Coklat.JPG: Prediksi = Kuning (100.00%)
Deteksi Objek pada kubus5/w/Hijau Muda.JPG: Prediksi = Putih (100.00%)
Deteksi Objek pada kubus5/w/Hijau Tua.JPG: Prediksi = Biru (100.00%)
Deteksi Objek pada kubus5/w/Kuning Muda.JPG: Prediksi = Putih (100.00%)
Deteksi Objek pada kubus5/w/Orange.JPG: Prediksi = Merah (100.00%)
Deteksi Objek pada kubus5/w/Pink.JPG: Prediksi = Merah (100.00%)
Deteksi Objek pada kubus5/w/Ungu.JPG: Prediksi = Merah (100.00%)


In [87]:
# Hitung dan cetak akurasi keseluruhan
accuracy_percentage = (accuracy / total_images ) # Use the total number of correct predictions instead of the last accuracy score
print(f"\nAkurasi Keseluruhan: {accuracy_percentage:.2f}% dari {total_images} gambar yang diproses.")


Akurasi Keseluruhan: 11.11% dari 9 gambar yang diproses.
